In [8]:
import requests
import datetime
from datetime import datetime as dt
import time
from bs4 import BeautifulSoup

In [5]:
points = {
    "Antsahamamy":[-18.92, 47.56],
    "Ambohimiarina":[-18.83, 47.13],
    "Ambohitsilaozana":[-17.70, 48.47],
    "Ambongabe":[-18.53, 48.03],
    "Ampitatsimo":[-17.81, 48.38],
}

contact_email = "jeremy.lavarenne@cirad.fr"
TAMSAT_batch_ref = "tamsat_subset_707"
date_start = datetime.datetime(2020,1,1,0,0)
date_end = datetime.datetime(2021,1,1,0,0)

In [6]:
# pass all requests
url = 'http://www.tamsat.org.uk/data-subset/data'


for point in points :

	print("point",point)

	params={
		"dataset": "01-tamsatDaily.v3.1",
		"datatype": "point",
		"lat": str(points[point][0]),
		"lon": str(points[point][1]),
		"zone": "BOUNDS",
		"maxLat": "10",
		"minLon": "-10",
		"maxLon": "10",
		"minLat": "-10",
		"starttime": date_start.isoformat()+("Z"),
		"endtime": date_end.isoformat()+("Z"),
		"email": contact_email,
		"ref": TAMSAT_batch_ref,
	}

	# passing request
	response = requests.request("POST", url, data=params)

	if response.status_code == 200:
		print("HTTP response OK")
	else :
		print("HTTP response not OK")

	# # if request not OK, waiting and retrying
	# retries = 0
	# max_retries = 5

	# while response.status_code != 200 :
	# 	retries = retries + 1
	# 	print('request not OK, waiting 30s and retrying (',retries,"/",max_retries)
	# 	time.sleep(30)
	# 	print("retrying...")
	# 	response = requests.request("GET", url, data=params)
	# 	if retries == max_retries :
	# 		print("max retries reached. breaking for point",point)
	# 		break

	print("waiting 10s before passing next request...")
	time.sleep(10)

point Antsahamamy
HTTP response OK
waiting 10s before passing next request...
point Ambohimiarina
HTTP response OK
waiting 10s before passing next request...
point Ambohitsilaozana
HTTP response OK
waiting 10s before passing next request...
point Ambongabe
HTTP response OK
waiting 10s before passing next request...
point Ampitatsimo
HTTP response OK
waiting 10s before passing next request...


In [9]:
# retrieving links to files
base_url = "http://www.tamsat.org.uk/data-subset/"
urls = 'http://www.tamsat.org.uk/data-subset/data?email='+contact_email+'&ref='+TAMSAT_batch_ref
links = []

while len(links) != len(points)+1 :
   grab = requests.get(urls)
   soup = BeautifulSoup(grab.text, 'html.parser')

   # traverse paragraphs from soup
   links = []

   for link in soup.find_all("a"):
      data = link.get('href')
      links = links + [data]

   print("Waiting for data to be ready before download (",len(links)-1,"/",len(points),"). Refreshing in 60s...")
   time.sleep(60)


Waiting for data to be ready before download ( 5 / 5 ). Refreshing in 60s...


In [22]:
save_path = "../data/3_output/"

for i in range(len(points)):
    link = links[i]
    URL = base_url+"data/"+link
    response = requests.get(URL)
    save_filename = "TAMSAT_"+list(points.keys())[i]+"_"+''.join(links[i].split("-")[1:-1])
    open(os.path.join(save_path,save_filename), "wb").write(response.content)